# Setup references

references are added to the Hiperspace and Mermaid nuget packages, together with `CousinProblem.dll` (compiled with `dotnet build` in the download directory)

For this simple example, we're using the ` HeapSpace` driver (intended for mock and session usage), but the ` Hiperspace.Rocks` driver would be used for massive datasets

In [8]:
#r "nuget: Hiperspace"
#r "nuget: Hiperspace.Heap"
#r "./bin/debug/net8.0/CousinProblem.dll"
#r "nuget: Microsoft.DotNet.Interactive.Mermaid, 1.0.0-beta.24229.4"
;;


# Setup sample data
The same Sample Family tree from test cases, converted to F# syntax

![Model](cousins-white.svg "Title")


In [10]:
open Hiperspace
open Hiperspace.Heap
open Cousins
open Microsoft.DotNet.Interactive.Mermaid

let driver = new HeapSpace()
let space = new CousinsSpace (driver)

let tree = 
  [
    Person (Name = "Eve", Gender = Gender.Female);
    Person (Name = "Jane", Gender = Gender.Female, Mother =  Person (Name = "Eve" ));
    Person (Name = "Jack", Gender = Gender.Male, Mother =  Person (Name = "Eve" ));
    Person (Name = "Adam", Gender = Gender.Male, Father =  Person (Name = "Jack" ));
    Person (Name = "Liz", Gender = Gender.Female, Father =  Person (Name = "Adam" ));
    Person (Name = "Mary", Gender = Gender.Female, Mother =  Person (Name = "Jane" ));
    Person (Name = "Burt", Gender = Gender.Male, Mother=  Person (Name = "Jane" ));
    Person (Name = "Rob", Gender = Gender.Male, Father =  Person (Name = "Burt" ));
    Person (Name = "John", Gender = Gender.Male);
    Person (Name = "Lucy", Gender = Gender.Female, Father =  Person (Name = "John" ), Mother =  Person (Name = "Mary" ));
    Person (Name = "Mark", Gender = Gender.Male, Father =  Person (Name = "John" ), Mother =  Person (Name = "Mary"));
  ]

tree
|> List.iter (fun i -> space.Persons.Bind (i, false) |> ignore)
;;

# simple graph

`space.Nodes` is a view of all entity / segment / aspect that provide a mapping to the view. `nodes` a translation to Mermaid markup syntax,
`space.Edges` is a view of all entity / segment / aspect that provide a mapping to the Edge view.  In this case, four elements per person (via `Edge2/3/4` intermediate views)

```
entity Cousins.Person
    = Node (),
      Edge  (From = this,   To = Mother, Name = Name, TypeName = "Mother"),
      Edge2 (From = this,   To = Father, Name = Name, TypeName = "Father"),
      Edge3 (From = Father, To = this,   Name = Name, TypeName = "Child"),
      Edge4 (From = Mother, To = this,   Name = Name, TypeName = "Child")
```

When viewed as a graph, edges projected for Mother/Father and  Child relations

The content is then rendered using  Mermaid visualization code

In [13]:
let strip (s : string) = s.Replace("=","")

let nodes = 
    space.Nodes 
    |> Seq.map      (fun n -> $"    {strip n.SKey}(\"{n.Name} ({n.TypeName})\")\n")
    |> Seq.fold     (fun a i  -> a + i) ""
let edges = 
    space.Edges 
    |> Seq.filter   (fun e -> not (e.From = null || e.To = null))
    |> Seq.map      (fun e -> $"    {strip e.From.SKey} -- {e.TypeName} --> {strip e.To.SKey}\n")
    |> Seq.fold     (fun a i -> a + i ) ""

let mermaid = $"""
flowchart TD 
{nodes}
{edges}
"""
MermaidMarkdown (mermaid)


# inferred relations

[Cousins.hilang](./Cousins.hilang) includes the extension property ` Relatives   = relation (this)` that uses the `%function (unary, relation, Cousins.Helper.Relations, List<Path>); ` hilang declaration for the [Relations(Person person)][./Helper.cs#L14] function to recursively search for all relations.

**NB** the client application has no knowledge of *how* the `Relative` is calculated, just that it is.

In this example we retrieve the set of paths for each person (*excluding any null references*) and add to the `Paths` entity (that also provides a mapping to `Edge`)

Finally, we render the graph using Mermaid

##### This graph view demonstrates the common problem may graph visualizations : a spider-web of lines, but no clear view

In [14]:
let temp = new CousinsSpace(new GenerationSpace ([|new HeapSpace(); space|]))

let relatives (p : Path seq) =
    p
    |> List.ofSeq
    |> List.filter  (fun r -> not (r.From = null || r.To = null || r.TypeName = "relation" ))

space.Persons 
|> List.ofSeq
|> List.map         (fun p -> relatives p.Relatives)
|> List.fold        (fun a i -> i @ a) []
|> List.iter        (fun i -> temp.Paths.Add i |> ignore)

let edges2 = 
    temp.Edges
    |> List.ofSeq
    |> List.filter  (fun r -> not (r.From = null || r.To = null || r.TypeName = "relation" ))
    |> List.map     (fun e -> $"    {strip (e.From.SKey)} -- {e.TypeName} --> {strip (e.To.SKey)}\n")
    |> List.fold    (fun a i -> a + i ) ""

let mermaid = $"""
flowchart TD 
{nodes}
{edges2}
"""
MermaidMarkdown (mermaid)


# Just Lucy
Filtering the set of direct relations and inferred paths to those from one person, shows their view of relations inferred by parents.  An update to any part of the graph changes the inferred relations without complex recalculation

In [15]:
let lucys = 
    temp.Edges
    |> List.ofSeq
    |> List.filter  (fun r -> not (r.From = null || r.To = null))
    |> List.filter  (fun r -> r.From.Name = "Lucy")
    |> List.map     (fun e -> $"    {e.From.Name} -- {e.TypeName} --> {e.To.Name}\n")
    |> List.fold    (fun a i -> a + i ) ""

let mermaid = $"""
flowchart TD 
{lucys}
"""
MermaidMarkdown (mermaid)
